In [1]:
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import sys
import os
import copy

In [ ]:
# Set working directory
os.chdir(r"C:\Users\Lehnert Lab\GitHub\bcqt_hub\experiments\VNA Experiments\Fast Qi Tracking")

sys.path.append(r"C:\\Users\\Lehnert Lab\\GitHub")
sys.path.append(r"C:\\Users\\Lehnert Lab\\GitHub\\drivers")

import bcqt_hub
import bcqt_hub.bcqt_hub.drivers

# Import VNA driver
from bcqt_hub.bcqt_hub.drivers.instruments.VNA_Keysight import VNA_Keysight

# VNA Configuration
VNA_Keysight_InstrConfig = {
    "instrument_name": "VNA_Keysight",
    "rm_backend": None,
    "instr_address": 'TCPIP0::192.168.0.105::inst0::INSTR',
    "power": -80,
    "averages": 10,
    "sparam": ['S21'],
    "edelay": 71.8,
    "f_center": 1e9,
    "f_span": 100e6,
    "segment_type": "homophasal",
    "n_points": 41,
    "Noffres": 10,
    "if_bandwidth": 500,
}

# Initialize VNA
PNA_X = VNA_Keysight(VNA_Keysight_InstrConfig, debug=True)
all_dfs = {}

# Check instrument error queue and configure
PNA_X.check_instr_error_queue()
PNA_X.filter_configs()
PNA_X.setup_s2p_measurement()

# Display configurations
display(PNA_X.configs)

# Import data analysis tools
sys.path.append(r"C:\\Users\\Lehnert Lab\\GitHub\\bcqt_hub\\experiments")
from bcqt_hub.bcqt_hub.src.DataAnalysis import DataAnalysis
import bcqt_hub.experiments.quick_helpers as qh

# Import external attenuator
sys.path.append(r"C:\Users\Lehnert Lab\GitHub\bcqt_hub\bcqt_hub\drivers\misc\MiniCircuits")
sys.path.append(r"C:\Users\Lehnert Lab\GitHub\bcqt_hub\bcqt_hub\drivers\instruments")

# from MC_VarAttenuator import MC_VarAttenuator
# MC_Variable_Atten = MC_VarAttenuator("192.168.0.113")  


#### Helper functions for a single fast Qi measurement

In [3]:
def save_data_to_csv(s2p_df, csv_path, resonator_freq, configs, meas_idx, global_meas_idx, elapsed_time_sec):
    
    """
    Save measurement data to a CSV file with elapsed time in the filename.
    """
    
    resonator_freq, span, ifbw, avg, power = configs["f_center"], configs["f_span"], configs["if_bandwidth"], configs["averages"], configs["power"]
    freq_str = f"{resonator_freq/1e9:1.3f}".replace('.', "p")
    name = f"Global_Meas_{global_meas_idx}_Child_Meas_{meas_idx}_span_{span/1e3:1.0f}kHz_{freq_str}GHz_power_{power}dBm_{avg}_avgs_elapsed_{elapsed_time_sec:1.3f}sec"

    filename = str(csv_path / f"{name}.csv")
    s2p_df.to_csv(filename)
    
    print(f"File location: {filename}")

In [4]:
def analyze_data(dfs_in_progess, dstr, name, data_path):
    
    """
    Analyze measurement data and return fit parameters.
    """
    
    all_dfs_concat = pd.concat(dfs_in_progess)
    current_df = all_dfs_concat.groupby(all_dfs_concat.index).mean()
    Res_PowSweep_Analysis = DataAnalysis(current_df, dstr)

    try:
        params, conf_intervals, err, init1, fig = Res_PowSweep_Analysis.fit_single_res(
            data_df=current_df, save_dcm_plot=False, plot_title=name, save_path=data_path
        )
        return params, conf_intervals, fig
    
    except Exception as e:
        print(f"Error during analysis: {e}")
        return None, None, None

In [5]:
def calculate_qi_params(params, conf_intervals):
    
    """
    Calculate Qi and other parameters from fit results.
    """
    
    Q, Qc, phi = params["Q"], params["Qc"],params["phi"]
    Qi = 1 / (1 / Q - np.cos(phi) / np.abs(Qc))
    
    Q_err, Qi_err, Qc_err, Qc_Re_err, phi_err, f_center_err = conf_intervals

    perc_errs = {
        "Q_perc": abs(Q_err / Q * 100),
        "Qi_perc": abs(Qi_err / Qi * 100),
        "Qc_perc": abs(Qc_err / Qc * 100),
    }

    return perc_errs

#### Individual fast Qi measurement

In [11]:
def fast_qi_tracking(resonator_freq, power, averages=50, avg_step=25, max_run_time=5, 
                     err_threshold_perc=5, parent_start_time=None, save_path=None, 
                     global_meas_idx=0, ext_atten=0, dstr=""):
    
    """
    Perform fast Qi tracking for a single resonator frequency at a power.

    Args:
        resonator_freq (float): Resonator frequency to measure (in Hz).
        power (float): Power level for VNA measurements (in dBm).
        averages (int): Initial number of averages for measurements.
        avg_step (int): Step size to increase averages if error threshold is not met.
        max_run_time (float): Maximum time to run a single measurement (in minutes).
        err_threshold_perc (float): Error threshold for Qi (in percentage).
        parent_start_time (float): Start time of the parent function (in seconds since epoch).
        save_path (Path): Path to save CSV files.
        global_meas_idx (int): Global measurement index.
        ext_atten (float): External attenuation. (not used for now)
    """
    
    start_time = time.time()  # Start time of the child function
    max_end_time = start_time + max_run_time * 60  # Convert run_time to seconds

    configs = {
        "n_points": 201,
        "power": power,
        "averages": averages,
        "if_bandwidth": 1e3,
        "f_span": 0.4e6,
        "err_threshold_perc": err_threshold_perc,
        "Noffres": 10,
        "f_center": resonator_freq,
    }

    # Use VNA to take & download data
    # MC_Variable_Atten.Set_Attenuation(ext_atten)
    PNA_X.update_configs(**configs)
    configs["segments"] = PNA_X.compute_homophasal_segments()
    PNA_X.setup_s2p_measurement()

    meas_idx = 0
    Qi_err_perc = 100
    dfs_in_progess = []

    # Main measurement loop
    while time.time() < max_end_time and Qi_err_perc > configs["err_threshold_perc"]:
        
        print(f"Number of averages: {configs['averages']}")

        # Run measurement
        # PNA_X.update_configs(**configs)
        PNA_X.run_measurement()

        # Get data from VNA 
        s2p_df = PNA_X.return_data_s2p()
        
        title_str = f"{PNA_X.configs['f_span']/1e6:1.2f}MHz_span_{PNA_X.configs['averages']}_avgs_{PNA_X.configs['if_bandwidth']}_IFBW_{PNA_X.configs['power']}_dBm"
        
        # Decide whether to plot this measurement (e.g. Plot every 10 measurements)
        if (global_meas_idx % 10 == 0):
            axes = qh.plot_s2p_df(s2p_df)
            fig = axes[0][0].get_figure()
            fig.suptitle(title_str, size=16)
            fig.tight_layout()
            plt.show()

        dfs_in_progess.append(s2p_df)

        # Save data to CSV with elapsed time since parent function started
        elapsed_time_sec = (time.time() - parent_start_time) # Elapsed time in seconds (for better precision)
        save_data_to_csv(s2p_df, save_path, resonator_freq, configs, meas_idx, global_meas_idx, elapsed_time_sec)

        # Analyze data
        params, conf_intervals, fig = analyze_data(dfs_in_progess, dstr, title_str, save_path)
        if params is None:
            configs["averages"] += avg_step
            print(f"Retrying with {configs['averages']} averages.")
            continue

        # Calculate Qi and other parameters
        perc_errs = calculate_qi_params(params, conf_intervals)

        # Check if Qi_err is inf (because fit fails)
        if perc_errs["Qi_perc"] == np.inf:
            print(f"Fit fails: Qi_err = {perc_errs['Qi_perc'] = }")
            configs["averages"] += avg_step
            print(f"Retrying with {configs['averages']} averages.")
        else:
            Qi_err_perc = perc_errs["Qi_perc"]
            print(f"Current Qi err: {Qi_err_perc = }")

        meas_idx += 1

#### Test codes for individual fast Qi measurement (uncomment to run)

In [ ]:
dstr = datetime.today().strftime("%m_%d_%H%M")
current_dir = Path(".").absolute()
data_path = current_dir / "data" / dstr
data_path.mkdir(parents=True, exist_ok=True)

fast_qi_tracking(
                resonator_freq=5.081028e9,
                power=-75,
                averages=3000,
                avg_step=1500,
                max_run_time=10,  
                err_threshold_perc=6,
                parent_start_time=time.time(), 
                save_path=data_path,
                dstr=dstr
                )


#### Parent function to run multiple fast Qi measurements for multipole resonators at multipole powers

In [ ]:
def parent_function(resonator_freqs, power_config, run_time_mins, 
                    max_child_run_time_mins, err_threshold_perc=5):
    
    """
    Parent function to loop through a list of resonator frequencies and power configurations,
    and run the fast_qi_tracking function for each combination until run_time is reached.

    Args:
        resonator_freqs (list): List of resonator frequencies to measure (in Hz).
        power_config (dict): Dictionary defining power levels and their configurations.
        run_time_mins (float): Total time to run the parent function (in minutes).
        max_child_run_time_mins (float): Maximum time to run the child function (in minutes).
        err_threshold_perc (float): Error threshold for Qi (in percentage).
    """
    
    # Create main data folder
    dstr = datetime.today().strftime("%m_%d_%H%M")
    current_dir = Path(".").absolute()
    data_path = current_dir / "data" / dstr
    data_path.mkdir(parents=True, exist_ok=True)

    parent_start_time = time.time()  # Start time of the parent function
    parent_end_time = parent_start_time + run_time_mins * 60  # Convert run_time to seconds
    
    # Initialize global variable for measurement count 
    global_meas_idx = 0

    while time.time() < parent_end_time:
        
        for resonator_freq in resonator_freqs:
            
            # Create folder for the resonator frequency (in GHz)
            res_freq_ghz = f"{resonator_freq / 1e9:.3f}GHz"
            res_freq_path = data_path / res_freq_ghz
            res_freq_path.mkdir(parents=True, exist_ok=True)

            for power_name, power_settings in power_config.items():
                # Create folder for the power level
                power_folder = f"{power_settings['power']}dBm"
                power_path = res_freq_path / power_folder
                power_path.mkdir(parents=True, exist_ok=True)

                print(f"Starting measurement for resonator_freq = {res_freq_ghz}, {power_name} = {power_settings['power']} dBm")

                # Run the fast Qi tracking function for each resonator at a power
                fast_qi_tracking(
                    resonator_freq=resonator_freq,
                    power=power_settings["power"],
                    averages=power_settings["averages"],
                    avg_step=power_settings["avg_step"],
                    max_run_time=max_child_run_time_mins,  # This is the run_time for the child function
                    err_threshold_perc=err_threshold_perc,
                    parent_start_time=parent_start_time,  # Pass parent start time to child
                    save_path=power_path,  # Pass the save path to the child function
                    global_meas_idx=global_meas_idx,
                    ext_atten=power_settings["ext_atten"],
                    dstr=dstr
                )

                global_meas_idx += 1
                
                # Check if parent run_time has been reached
                if time.time() >= parent_end_time:
                    print("Parent function run_time reached. Stopping.")
                    return

        print("Completed one full cycle of resonator_freqs and powers. Repeating...")

    print("Parent function run_time reached. Stopping.")

#### Test codes for multipole fast Qi measuremensts (uncomment to run)

In [ ]:
# # Define resonator frequencies
# resonator_freqs = [4.363937e9]  # List of resonator frequencies in Hz

# # Define power configurations
# power_config = {
#     "high_power": {"power": -30, "averages": 5, "avg_step": 2, "ext_atten": 0},
#     # "medium_power": {"power": -50, "averages": 10, "avg_step": 5},
#     # "low_power": {"power": -75.5, "averages": 200, "avg_step": 100}
# }

# # Run the parent function
# parent_function(resonator_freqs, power_config, run_time_mins=0.5, max_child_run_time_mins=5)

#### Run measurement for real

In [ ]:
# MQC_BOE_SOLV1

# # Define resonator frequencies
# resonator_freqs = [4.7333838e9]  # List of resonator frequencies in Hz

# # Define power configurations
# power_config = {
#     "high_power": {"power": -30, "averages": 5, "avg_step": 2},
#     "medium_power": {"power": -50, "averages": 10, "avg_step": 5},
#     "low_power": {"power": -75.5, "averages": 200, "avg_step": 100}
# }

# # Run the parent function for 12 hours
# parent_function(resonator_freqs, power_config, run_time_mins=12*60, max_child_run_time_mins=5)

In [ ]:
# # MQC_BOE_02

# # Define resonator frequencies
# resonator_freqs = [4.736494e9]  # edelay: 62.14ns

# # Define power configurations (still need to test the number of avg)
# power_config = {
#     "high_power": {"power": -30, "averages": 5, "avg_step": 2, "ext_atten": 0},
#     "medium_power": {"power": -50, "averages": 60, "avg_step": 30, "ext_atten": 0},
#     "low_power": {"power": -70, "averages": 3000, "avg_step": 1500, "ext_atten": 0}
# }

# # Run the parent function for 12 hours
# parent_function(resonator_freqs, power_config, run_time_mins=16*60, max_child_run_time_mins=10, err_threshold_perc=6)

In [ ]:
# MQC_anneal_01

# Define resonator frequencies
resonator_freqs = [4.707753e9]  # edelay: 71.8 ns

# Define power configurations (still need to test the number of avg)
power_config = {
    "high_power": {"power": -35, "averages": 10, "avg_step": 5, "ext_atten": 0},
    "medium_power": {"power": -55, "averages": 100, "avg_step": 50, "ext_atten": 0},
    "low_power": {"power": -75, "averages": 2000, "avg_step": 1000, "ext_atten": 0}
}

# Run the parent function for 20 hours
parent_function(resonator_freqs, power_config, run_time_mins=20*60, max_child_run_time_mins=10, err_threshold_perc=6)